In [1]:
import json
import json
import os


In [2]:
pip install -U negate

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 769.7/769.7 kB 12.6 MB/s eta 0:00:00a 0:00:01
  Using cached typer-0.9.0-py3-none-any.whl (45 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.6/181.6 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 66.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.3/46.3 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.2/157.2 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 490.9/490.9 kB 31.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 920.1/920.1 kB 55.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 43.1 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45

In [3]:
from negate import Negator

# Use default model (en_core_web_md):
negator = Negator()

INFO: Downloading model. This only needs to happen once. Please, be patient...


In [4]:
# Define a function to perform 'and' and 'or' operations on two answers.
def and_operation(answer1, answer2):
    return 'yes' if answer1 == 'yes' and answer2 == 'yes' else 'no'

def or_operation(answer1, answer2):
    return 'yes' if answer1 == 'yes' or answer2 == 'yes' else 'no'

# Define a function to negate the answer
def negate_answer(answer):
    return 'no' if answer == 'yes' else 'yes'

In [7]:
out_path = "/scratch/nmachav1/MLLM_Hallucinations_CLEVR/dataset/language_augmentation/"
path = "/scratch/nmachav1/MLLM_Hallucinations_CLEVR/dataset/object_number_split/"


for filename in os.listdir(path):
    print(filename)
    if filename.endswith('.json'):
        file_path = os.path.join(path, filename)
        res_file_path = out_path + filename[:-5]
                    
        all_orig_data = []
        all_and_data = []
        all_or_data = []
        all_not_data = []
        all_complex_data = []
    
        with open(file_path, 'r') as f:
            data = json.load(f)       
            questions = [item for item in data if item.get("answer_type") == "yes-no"]        

            #with open(f'{res_file_path}_orig.json', 'a') as orig_file, open(f'{res_file_path}_and.json', 'a') as and_file, open(f'{res_file_path}_or.json', 'a') as or_file, open(f'{res_file_path}_not.json', 'a') as not_file, open(f'{res_file_path}_complex.json', 'a') as complex_file:     
            for i in range(0,len(questions)):
                image_questions_orig = []
                image_questions_and = []
                image_questions_or = []
                image_questions_not = []
                image_questions_complex = []
                #print(questions[i]['image_filename'], questions[i+1]['image_filename'])
                if i < (len(questions)-1) and questions[i]['image_filename'] != questions[i+1]['image_filename']:
                    continue
                elif i < (len(questions)-1):
                    question1 = questions[i]
                    question2 = questions[i+1]
                    question1_text = question1['question'][:-1]  # Remove the question mark at the end
                    question2_text = question2['question'][0].lower() + question2['question'][1:]
            
                    negated_question1 = negator.negate_sentence(question1['question'])
                    negated_question2 = negator.negate_sentence(question2['question'])
            
                    negated_question1_text = negated_question1[:-1]
                    negated_question2_text = negated_question2[0].lower() + negated_question2[1:]
            
                    answer1 = question1['ground_truth']
                    answer2 = question2['ground_truth']
            
                    negated_answer1 = negate_answer(answer1)
                    negated_answer2 = negate_answer(answer2)
            
                    answer_and = and_operation(answer1, answer2)
                    answer_or = or_operation(answer1, answer2)
                    answer_and_negated = and_operation(negated_answer1, negated_answer2)
                    answer_or_negated = or_operation(negated_answer1, negated_answer2)
        
                    image_questions_orig.append({
                        'image_filename': question1['image_filename'],
                        'question': question1['question'],
                        'q1 answer': answer1,
                        'q2 answer': "NA",
                        'ground_truth': question1['ground_truth'],
                        'model_generated_answer': "",
                        'consistent':""                       
                    })
                    image_questions_orig.append({
                        'image_filename': question1['image_filename'],
                        'question': question2['question'],
                        'q1 answer': answer2,
                        'q2 answer': "NA",
                        'ground_truth': question2['ground_truth'],
                        'model_generated_answer': "",
                        'consistent':""
                    })
                    image_questions_not.append({
                        'image_filename': question1['image_filename'],
                        'question': negated_question1,
                        'q1 answer': answer1,
                        'q2 answer': "NA",
                        'ground_truth': negated_answer1,
                        'model_generated_answer': "",
                        'consistent':""
                    })
                    image_questions_not.append({
                        'image_filename': question1['image_filename'],
                        'question': negated_question2,
                        'q1 answer': answer2,
                        'q2 answer': "NA",
                        'ground_truth': negated_answer2,
                        'model_generated_answer': "",
                        'consistent':""
                    })
                    image_questions_and.append({
                        'image_filename': question1['image_filename'],
                        'question': f"{question1_text} and {question2_text}",
                        'q1 answer': answer1,
                        'q2 answer': answer2,
                        'ground_truth': answer_and,
                        'model_generated_answer': "",
                        'consistent':""
                    })
                    image_questions_or.append({
                        'image_filename': question1['image_filename'],
                        'question': f"{question1_text} or {question2_text}",
                        'q1 answer': answer1,
                        'q2 answer': answer2,
                        'ground_truth': answer_or,
                        'model_generated_answer': "",
                        'consistent':""
                    })
                    image_questions_complex.append({
                        'image_filename': question1['image_filename'],
                        'question': f"{question1_text} and {negated_question2_text}",
                        'q1 answer': answer1,
                        'q2 answer': negated_answer2,
                        'ground_truth': answer_and_negated,
                        'model_generated_answer': "",
                        'consistent':""
                    })
                    image_questions_complex.append({
                        'image_filename': question1['image_filename'],
                        'question': f"{question1_text} or {negated_question2_text}",
                        'q1 answer': answer1,
                        'q2 answer': negated_answer2,
                        'ground_truth': answer_or_negated,
                        'model_generated_answer': "",
                        'consistent':""
                    })
                    image_questions_complex.append({
                        'image_filename': question1['image_filename'],
                        'question': f"{negated_question1_text} and {question2_text}",
                        'q1 answer': negated_answer1,
                        'q2 answer': answer2,
                        'ground_truth': answer_and_negated,
                        'model_generated_answer': "",
                        'consistent':""
                    })
                    image_questions_complex.append({
                        'image_filename': question1['image_filename'],
                        'question': f"{negated_question1_text} or {question2_text}",
                        'q1 answer': negated_answer1,
                        'q2 answer': answer2,
                        'ground_truth': answer_or_negated,
                        'model_generated_answer': "",
                        'consistent':""
                    })
                    all_orig_data.extend(image_questions_orig)
                    all_and_data.extend(image_questions_and)
                    all_or_data.extend(image_questions_or)
                    all_not_data.extend(image_questions_not)
                    all_complex_data.extend(image_questions_complex)


        with open(f'{res_file_path}_orig.json', 'a') as orig_file, open(f'{res_file_path}_and.json', 'a') as and_file, open(f'{res_file_path}_or.json', 'a') as or_file, open(f'{res_file_path}_not.json', 'a') as not_file, open(f'{res_file_path}_complex.json', 'a') as complex_file:     
            json.dump(all_orig_data, orig_file, indent=4)
            json.dump(all_and_data, and_file, indent=4)
            json.dump(all_or_data, or_file, indent=4)
            json.dump(all_not_data, not_file, indent=4)
            json.dump(all_complex_data, complex_file, indent=4)

.ipynb_checkpoints
val_num_10.json
val_num_6.json
val_num_7.json
val_num_5.json
val_num_9.json
val_num_4.json
val_num_8.json
val_num_3.json


In [21]:
for i in range(0,len(questions),2):
    
    print(i)
    i = i-1

0
2
4
6
8
10
12
14
16
18
20
22
24
26
28
30
32
34
36
38
40
42
44
46
48
50
52
54
56
58
60
62
64
66
68
70
72
74
76
78
80
82
84
86
88
90
92
94
96
98
100
102
104
106
108
110
112
114
116
118
120
122
124
126
128
130
132
134
136
138
140
142
144
146
148
150
152
154
156
158
160
162
164
166
168
170
172
174
176
178
180
182
184
186
188
190
192
194
196
198
200
202
204
206
208
210
212
214
216
218
220
222
224
226
228
230
232
234
236
238
240
242
244
246
248
250
252
254
256
258
260
262
264
266
268
270
272
274
276
278
280
282
284
286
288
290
292
294
296
298
300
302
304
306
308
310
312
314
316
318
320
322
324
326
328
330
332
334
336
338
340
342
344
346
348
350
352
354
356
358
360
362
364
366
368
370
372
374
376
378
380
382
384
386
388
390
392
394
396
398
400
402
404
406
408
410
412
414
416
418
420
422
424
426
428
430
432
434
436
438
440
442
444
446
448
450
452
454
456
458
460
462
464
466
468
470
472
474
476
478
480
482
484
486
488
490
492
494
496
498
500
502
504
506
508
510
512
514
516
518
520
522
524
526